<a href="https://colab.research.google.com/github/coll-j/indonesia-locations-data/blob/main/Parser_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloads, Installs, Imports, and File names

In [ ]:
!wget https://raw.githubusercontent.com/pararawendy/border-desa-indonesia-geojson/master/indonesia_villages_border.geojson.zip

In [ ]:
!unzip indonesia_villages_border.geojson.zip

In [ ]:
!wget https://raw.githubusercontent.com/benangmerah/wilayah/master/datasources/daftar-nama-daerah.csv

In [ ]:
!pip install ijson

In [16]:
import pandas as pd
import ijson # large json file parser
import numpy as np
import re
import json
from itertools import zip_longest
from google.colab import files

JSON_TO_CSV_INPUT_PATH = './indonesia_villages_border.geojson'
JSON_TO_CSV_OUT_PATH = 'Village_LongLat_Approx.csv'

PROV_KOTA_INPUT_1_PATH = './daftar-nama-daerah.csv'
PROV_KOTA_INPUT_2_PATH = './list_location_id_csv.csv'

PROV_INPUT_PATH = './provinces.csv'
PROV_OUT_PATH = 'provinces_lat_long.csv'

KOTA_INPUT_PATH = './kota_kab.csv'
KOTA_OUPUT_PATH = 'kota_kab_lat_long.csv'

KEC_INPUT_PATH = './kecamatan.csv'
KEC_OUTPUT_PATH = 'kecamatan_lat_long.csv'

KEL_INPUT_PATH = 'kelurahan.csv'
KEL_OUTPUT_PATH = 'kelurahan_lat_long.csv'

# Parse Json into csv

In [ ]:
# test is an example of data we acquired from the geojson file
test = [[[109.3062, -0.4433, ], [109.3037, -0.4409, ], [109.2975, -0.4374, ], [109.2955, -0.434, ], [109.2921, -0.4313, ], [109.2899, -0.4291, ], [109.2801, -0.4268, ], [109.2781, -0.4265, ], [109.2742, -0.4346, ], [109.2696, -0.4429, ], [109.2692, -0.4488, ], [109.2693, -0.4492, ], [109.2695, -0.45, ], [109.2583, -0.4657, ], [109.2581, -0.4659, ], [109.2483, -0.4724, ], [109.2411, -0.4768, ], [109.2313, -0.483, ], [109.2293, -0.4856, ], [109.2273, -0.4882, ], [109.2327, -0.4974, ], [109.2533, -0.5022, ], [109.2839, -0.5122, ], [109.2897, -0.5132, ], [109.2899, -0.5134, ], [109.2919, -0.5153, ], [109.2923, -0.5161, ], [109.2934, -0.5173, ], [109.2935, -0.5176, ], [109.2977, -0.5214, ], [109.3009, -0.5249, ], [109.3284, -0.555, ], [109.3291, -0.5558, ], [109.3302, -0.557, ], [109.3322, -0.5557, ], [109.3337, -0.5546, ], [109.334, -0.5534, ], [109.3344, -0.553, ], [109.3351, -0.5512, ], [109.3362, -0.5496, ], [109.3368, -0.5482, ], [109.3379, -0.5468, ], [109.3391, -0.5456, ], [109.3399, -0.5443, ], [109.34, -0.5431, ], [109.3399, -0.5421, ], [109.3395, -0.5407, ], [109.339, -0.5399, ], [109.3385, -0.5387, ], [109.3379, -0.5377, ], [109.3378, -0.5364, ], [109.3378, -0.5344, ], [109.3382, -0.5327, ], [109.3386, -0.5312, ], [109.3394, -0.5296, ], [109.3402, -0.5284, ], [109.341, -0.5274, ], [109.3419, -0.5264, ], [109.3431, -0.5253, ], [109.3456, -0.5236, ], [109.3473, -0.5224, ], [109.3486, -0.5213, ], [109.3499, -0.5201, ], [109.3512, -0.5188, ], [109.3523, -0.5179, ], [109.3539, -0.5155, ], [109.3548, -0.5146, ], [109.3564, -0.5132, ], [109.3573, -0.5121, ], [109.358, -0.5111, ], [109.3589, -0.5104, ], [109.3598, -0.5097, ], [109.3609, -0.5091, ], [109.3627, -0.5086, ], [109.3642, -0.5082, ], [109.3648, -0.508, ], [109.365, -0.508, ], [109.366, -0.5071, ], [109.3694, -0.5029, ], [109.3707, -0.5009, ], [109.3721, -0.4994, ], [109.3737, -0.498, ], [109.3751, -0.4963, ], [109.3759, -0.4951, ], [109.3767, -0.4936, ], [109.3767, -0.4923, ], [109.3766, -0.491, ], [109.376, -0.4893, ], [109.3756, -0.4879, ], [109.3748, -0.4862, ], [109.3746, -0.4853, ], [109.3738, -0.4838, ], [109.3734, -0.4829, ], [109.373, -0.482, ], [109.3726, -0.4813, ], [109.3713, -0.4824, ], [109.3693, -0.4803, ], [109.3665, -0.4837, ], [109.3624, -0.4851, ], [109.3631, -0.481, ], [109.3665, -0.4693, ], [109.3659, -0.4652, ], [109.3631, -0.4652, ], [109.3576, -0.4693, ], [109.3563, -0.469, ], [109.3549, -0.4686, ], [109.3508, -0.4631, ], [109.343, -0.4536, ], [109.3418, -0.4522, ], [109.3341, -0.4457, ], [109.3254, -0.4439, ], [109.3151, -0.4453, ], [109.3082, -0.4453, ], [109.3062, -0.4433, ], ], [[109.2703, -0.4699, ], [109.2887, -0.4466, ], [109.3307, -0.4685, ], [109.3113, -0.4919, ], [109.2703, -0.4699, ], ], ]

def dim(a):
    if not type(a) == list:
        return []
    return [len(a)] + dim(a[0])

dim(test)

[2, 114, 2]

In [ ]:
def reduce_dim(array_3d): # some data are formed in a 3d array, so we need to reduce it into 2d array
  out = []
  for x in array_3d:
    for y in x:
      print(y)
      out.append([97.3112, 3.1187])

  return out

out = reduce_dim(test)
dim(out)

In [ ]:
data = ijson.parse(open(JSON_TO_CSV_INPUT_PATH, 'r'))
json_str = ''
start = True
in_array = 0

df = pd.DataFrame(columns=['province', 'district', 'sub_district', 'village', 'approx_long', 'approx_lat'])

def get_str(event, value, in_array):
  # get the value given the event
  # ex: if the event is start_map then it's a start of a dict
  # hence, returns '{'
  return{
      'start_map': '{',
      'end_map': '}',
      'start_array': '[' if in_array > 0 else '"[', # if it's the beginning of an array, we need to add '"' (json format)
      'end_array': '], ' if in_array > 0 else ']"',
      'map_key': '"{}": '.format(value),
      'string': '"{}", '.format(value),
      'number': '{}, '.format(value),
  }[event]
for prefix, event, value in data:
  if start: # ignore the first '['
    start = False
    continue

  if event == 'end_array':
    in_array -= 1
  if value is not None and event != 'number':
    value = re.sub('[\"]', '', value)
  json_str += get_str(event, value, in_array)
  if event == 'start_array':
    in_array += 1
  if event == 'end_map':
    try:
      row = json.loads(json_str)
      coords = eval(row['border'])
      if len(dim(coords)) > 2:
        coords = reduce_dim(coords)
      approx_long_lat = np.array(coords).mean(axis=0)
      row['approx_long'] = approx_long_lat[0]
      row['approx_lat'] = approx_long_lat[1]
      del row['border']
      df = df.append(row, ignore_index=True)
    except Exception as e:
      # debugging
      print(json_str)
      print(str(e))
      break

    json_str = ''
    try:
      del row
    except:
      pass

df.head()

In [ ]:
df

,province,district,sub_district,village,approx_long,approx_lat
0,ACEH,ACEH SELATAN,BAKONGAN,KEUDE BAKONGAN,97.475542,2.928458
1,ACEH,ACEH SELATAN,BAKONGAN,UJUNG MANGKI,97.446373,2.941181
2,ACEH,ACEH SELATAN,BAKONGAN,UJUNG PADANG,97.493403,2.941533
3,ACEH,ACEH SELATAN,BAKONGAN,KAMPONG DRIEN,97.478184,2.977676
4,ACEH,ACEH SELATAN,BAKONGAN,DARUL IKHSAN,97.418400,2.956487
...,...,...,...,...,...,...
83317,PAPUA BARAT,KOTA SORONG,MALAIMSIMSA,KLABULU,131.306823,-0.870793
83318,PAPUA BARAT,KOTA SORONG,MALADUM MES,SUPRAU,131.286656,-0.816865
83319,PAPUA BARAT,KOTA SORONG,MALADUM MES,TAMPA GARAM,131.253431,-0.819254
83320,PAPUA BARAT,KOTA SORONG,MALADUM MES,SAOKA,131.352492,-0.794798


In [ ]:
df.to_csv(JSON_TO_CSV_OUT_PATH) # save the file

# Provinsi

In [ ]:
df = pd.read_csv(PROV_KOTA_INPUT_PATH)

df.head()

,nid,parent_nid,name,serial,type,latitude,longitude,status
0,1,0,Provinsi Aceh,11,1,4.695135,96.749399,1
1,2,0,Provinsi Sumatera Utara,12,1,2.115355,99.545097,1
2,3,0,Provinsi Sumatera Barat,13,1,-0.739940,100.800005,1
3,4,0,Provinsi Riau,14,1,0.293347,101.706829,1
4,5,0,Provinsi Jambi,15,1,-1.485183,102.438058,1


In [ ]:
# turn it into a dict for easy access
dict_data = {}
for index, row in df.iterrows():
  if re.search('^Provinsi *', row['name']):
    dict_data[row['name'][9:].upper()] = { 'lat': row['latitude'], 'long': row['longitude']}

dict_data

{'ACEH': {'lat': 4.695135, 'long': 96.74939930000001},
 'BALI': {'lat': -8.409517800000001, 'long': 115.18891599999999},
 'BANTEN': {'lat': -6.4058171999999995, 'long': 106.0640179},
 'BENGKULU': {'lat': -3.5778470999999996, 'long': 102.3463875},
 'DI YOGYAKARTA': {'lat': -7.8753849, 'long': 110.4262088},
 'DKI JAKARTA': {'lat': -6.211544, 'long': 106.84517199999999},
 'GORONTALO': {'lat': 0.6999371999999999, 'long': 122.44672379999999},
 'JAMBI': {'lat': -1.4851831, 'long': 102.43805809999999},
 'JAWA BARAT': {'lat': -7.090911, 'long': 107.66888700000001},
 'JAWA TENGAH': {'lat': -7.150975, 'long': 110.1402594},
 'JAWA TIMUR': {'lat': -7.536063900000001, 'long': 112.2384017},
 'KALIMANTAN BARAT': {'lat': -0.2787808, 'long': 111.47528510000001},
 'KALIMANTAN SELATAN': {'lat': -3.0926415, 'long': 115.28375849999999},
 'KALIMANTAN TENGAH': {'lat': -1.6814878, 'long': 113.38235449999999},
 'KALIMANTAN TIMUR': {'lat': 1.6406296000000002, 'long': 116.419389},
 'KEPULAUAN BANGKA BELITUNG': {

In [ ]:
df_prov = pd.read_csv(PROV_INPUT_PATH)
df_prov['lat'] = np.nan
df_prov['long'] = np.nan

df_prov.head()

,id,name,lat,long
0,11,ACEH,NaN,NaN
1,12,SUMATERA UTARA,NaN,NaN
2,13,SUMATERA BARAT,NaN,NaN
3,14,RIAU,NaN,NaN
4,15,JAMBI,NaN,NaN


In [ ]:
for index, row in df_prov.iterrows():
  if row['name'] in dict_data:
    df_prov.loc[index, 'lat'] = dict_data[row['name']]['lat']
    df_prov.loc[index, 'long'] = dict_data[row['name']]['long']

df_prov

,id,name,lat,long
0,11,ACEH,4.695135,96.749399
1,12,SUMATERA UTARA,2.115355,99.545097
2,13,SUMATERA BARAT,-0.739940,100.800005
3,14,RIAU,0.293347,101.706829
4,15,JAMBI,-1.485183,102.438058
5,16,SUMATERA SELATAN,-3.319437,103.914399
6,17,BENGKULU,-3.577847,102.346388
7,18,LAMPUNG,-4.558585,105.406808
8,19,KEPULAUAN BANGKA BELITUNG,-2.741051,106.440587
9,21,KEPULAUAN RIAU,3.945651,108.142867


In [ ]:
df_prov.to_csv(PROV_OUT_PATH)

# Kota

In [ ]:
df_1 = pd.read_csv(PROV_KOTA_INPUT_1_PATH)
df_2 = pd.read_csv(PROV_KOTA_INPUT_2_PATH)
df_final = pd.read_csv(KOTA_INPUT_PATH)

df_1.shape, df_2.shape, df_final.shape

((543, 8), (516, 3), (514, 3))

> We can see that our sources have different number of data. We will use both to ensure we got the most. Better to be safe than sorry :)

In [ ]:
df_1 = df_1[df_1['name'].str.contains("Provinsi") == False] # takes only kota (city) data
df_1.shape

(510, 8)

In [ ]:
df_1.head()

,nid,parent_nid,name,serial,type,latitude,longitude,status
33,562,74,Kota Tangerang Selatan,0,2,-6.288889,106.718056,1
34,821,15,Kabupaten Banyuwangi,1501,2,-8.218611,114.366944,1
35,823,15,Kabupaten Madiun,1502,2,-7.627753,111.505483,1
36,824,15,Kabupaten Ponorogo,1503,2,-7.867827,111.466003,1
37,825,15,Kabupaten Magetan,1504,2,-7.649341,111.338159,1


In [ ]:
df_2.head()

,kabko,lat,long
0,KAB. ACEH BARAT,4.4543,96.1527
1,KAB. ACEH BARAT DAYA,3.7963,97.0068
2,KAB. ACEH BESAR,5.4529,95.4778
3,KAB. ACEH JAYA,4.7874,95.6458
4,KAB. ACEH SELATAN,3.3115,97.3517


In [ ]:
df_final.head()

,id,foreign,name
0,1101,11,KABUPATEN SIMEULUE
1,1102,11,KABUPATEN ACEH SINGKIL
2,1103,11,KABUPATEN ACEH SELATAN
3,1104,11,KABUPATEN ACEH TENGGARA
4,1105,11,KABUPATEN ACEH TIMUR


In [ ]:
df_final['lat'] = np.nan
df_final['long'] = np.nan

df_final.head()

,id,foreign,name,lat,long
0,1101,11,KABUPATEN SIMEULUE,NaN,NaN
1,1102,11,KABUPATEN ACEH SINGKIL,NaN,NaN
2,1103,11,KABUPATEN ACEH SELATAN,NaN,NaN
3,1104,11,KABUPATEN ACEH TENGGARA,NaN,NaN
4,1105,11,KABUPATEN ACEH TIMUR,NaN,NaN


In [ ]:
q = ['KABUPATEN', 'KOTA']
count = 0
for index, row in df_final.iterrows():
  kota = re.findall(r"["+'|'.join(q)+r"] (.*)", row['name'])[0] # regex to parse the name of kota (city)
  try:
    select = df_1[df_1['name'].str.upper().str.contains(kota)].iloc[0]
    df_final.loc[index, 'lat'] = select['latitude']
    df_final.loc[index, 'long'] = select['longitude']
  except Exception as e:
    # if the first data doesn't have the current city, try to search it in the second one
    try:
      select = df_2[df_2['kabko'].str.upper().str.contains(kota)].iloc[0]
      df_final.loc[index, 'lat'] = select['lat']
      df_final.loc[index, 'long'] = select['long']
    except Exception as e:
      print(str(e))
      pass
df_final

In [ ]:
df_final.isna().sum()

id         0
foreign    0
name       0
lat        7
long       7
dtype: int64

In [ ]:
df_final.to_csv(, na_rep='null')

# Kecamatan

In [ ]:
df_kec_kel = pd.read_csv(JSON_TO_CSV_OUT_PATH)
df_kec = pd.read_csv(KEC_INPUT_PATH)
df_kec_kel.shape, df_kec.shape

((83322, 7), (7215, 3))

In [ ]:
df_kec.head()

,id,foreign,name
0,1101010,1101,TEUPAH SELATAN
1,1101020,1101,SIMEULUE TIMUR
2,1101021,1101,TEUPAH BARAT
3,1101022,1101,TEUPAH TENGAH
4,1101030,1101,SIMEULUE TENGAH


In [ ]:
df_kota = pd.read_csv(KOTA_INPUT_PATH)
df_kota.head()

,id,foreign,name
0,1101,11,KABUPATEN SIMEULUE
1,1102,11,KABUPATEN ACEH SINGKIL
2,1103,11,KABUPATEN ACEH SELATAN
3,1104,11,KABUPATEN ACEH TENGGARA
4,1105,11,KABUPATEN ACEH TIMUR


> Since there are some kecamatan (districts) that have the same name in different cities and the `json_to_csv` file contains the name of the city, it's better if we have the parent (city) name in the output dataframe.

In [ ]:
def getBaseNama(row, col): # get the city name
  q = ['KABUPATEN', 'KOTA']
  try:
    return re.findall(r"["+'|'.join(q)+r"] (.*)", row[col])[0]
  except:
    return row[col]

In [ ]:
df_kota.tail()

,id,foreign,name,basename
509,9433,94,KABUPATEN PUNCAK,PUNCAK
510,9434,94,KABUPATEN DOGIYAI,DOGIYAI
511,9435,94,KABUPATEN INTAN JAYA,INTAN JAYA
512,9436,94,KABUPATEN DEIYAI,DEIYAI
513,9471,94,KOTA JAYAPURA,JAYAPURA


In [ ]:
def getParentName(row, col, df_parent): # get the parent name
  return df_parent[df_parent['id'] == row['foreign']].iloc[0][col]
df_kec['foreign_nama'] = df_kec.apply(lambda row: getParentName(row, 'basename', df_kota), axis=1)
df_kec.head()

,id,foreign,name,lat,long,foreign_nama
0,1101010,1101,TEUPAH SELATAN,2.464285,96.520430,SIMEULUE
1,1101020,1101,SIMEULUE TIMUR,2.640112,96.580326,SIMEULUE
2,1101021,1101,TEUPAH BARAT,2.554213,96.349640,SIMEULUE
3,1101022,1101,TEUPAH TENGAH,2.411233,96.334985,SIMEULUE
4,1101030,1101,SIMEULUE TENGAH,2.645959,96.090894,SIMEULUE


> So the idea was to approximate the kecamatan (districts) data based on the mean of the kelurahan (villages) lat long. Just like what we did when parsing the lat long from boundaries.

In [ ]:
mean_kec = df_kec_kel.groupby(['district', 'sub_district'])[['approx_long', 'approx_lat']].mean() # get the mean of kelurahan
mean_kec.reset_index(inplace=True)
mean_kec.head()

,district,sub_district,approx_long,approx_lat
0,ACEH BARAT,ARONGAN LAMBALEK,95.953161,4.328265
1,ACEH BARAT,BUBON,96.116779,4.304903
2,ACEH BARAT,JOHAN PAHWALAN,96.125292,4.159176
3,ACEH BARAT,KAWAY XVI,96.194050,4.266901
4,ACEH BARAT,MEUREUBO,96.185277,4.164602


In [ ]:
df_kec['lat'] = np.nan
df_kec['long'] = np.nan
df_kec[df_kec['foreign_nama'] == 'JAYAPURA'] # take little look inside the dataframe

,id,foreign,name,foreign_nama,lat,long
6707,9403080,9403,KAUREH,JAYAPURA,NaN,NaN
6708,9403081,9403,AIRU,JAYAPURA,NaN,NaN
6709,9403082,9403,YAPSI,JAYAPURA,NaN,NaN
6710,9403140,9403,KEMTUK,JAYAPURA,NaN,NaN
6711,9403150,9403,KEMTUK GRESI,JAYAPURA,NaN,NaN
6712,9403151,9403,GRESI SELATAN,JAYAPURA,NaN,NaN
6713,9403160,9403,NIMBORAN,JAYAPURA,NaN,NaN
6714,9403161,9403,NIMBORAN TIMUR / NAMBLONG,JAYAPURA,NaN,NaN
6715,9403170,9403,NIMBOKRANG,JAYAPURA,NaN,NaN
6716,9403180,9403,UNURUM GUAY,JAYAPURA,NaN,NaN


In [ ]:
# some district names contain 'Kota' or 'Kabupaten' string inside it. ex: Kota Jayapura
# to ensure the uniformity, we parse the base name instead
mean_kec['district'] = mean_kec.apply(lambda row: getBaseNama(row, 'district'), axis=1)
mean_kec[(mean_kec['district'] == 'JAYAPURA') & (mean_kec['sub_district'] == 'AIRU')]

In [ ]:
# get the lat long data
for index, row in df_kec.iterrows():
  try:
    select = mean_kec[(mean_kec['district'] == row['foreign_nama']) & (mean_kec['sub_district'] == row['name'])].iloc[0]
    df_kec.loc[index, 'lat'] = select['approx_lat']
    df_kec.loc[index, 'long'] = select['approx_long']
  except:
    pass

df_kec.head()

,id,foreign,name,foreign_nama,lat,long
0,1101010,1101,TEUPAH SELATAN,SIMEULUE,2.464285,96.520430
1,1101020,1101,SIMEULUE TIMUR,SIMEULUE,2.640112,96.580326
2,1101021,1101,TEUPAH BARAT,SIMEULUE,2.554213,96.349640
3,1101022,1101,TEUPAH TENGAH,SIMEULUE,2.411233,96.334985
4,1101030,1101,SIMEULUE TENGAH,SIMEULUE,2.645959,96.090894


In [ ]:
del df_kec['foreign_nama'] # delete the foreign nama

In [ ]:
df_kec.head()

,id,foreign,name,lat,long
0,1101010,1101,TEUPAH SELATAN,2.464285,96.520430
1,1101020,1101,SIMEULUE TIMUR,2.640112,96.580326
2,1101021,1101,TEUPAH BARAT,2.554213,96.349640
3,1101022,1101,TEUPAH TENGAH,2.411233,96.334985
4,1101030,1101,SIMEULUE TENGAH,2.645959,96.090894


In [ ]:
df_kec.to_csv(KEC_OUTPUT_PATH, na_rep='null')
files.download(KEC_OUTPUT_PATH)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Cleaning Kelurahan

In [ ]:
df_kel = pd.read_csv(KEL_INPUT_PATH)
df_kel.head()

,id,foreign,name
0,1101010001,1101010,LATIUNG
1,1101010002,1101010,LABUHAN BAJAU
2,1101010003,1101010,SUAK LAMATAN
3,1101010004,1101010,ANA AO
4,1101010005,1101010,LATALING


In [ ]:
df_kec_kel.head()

,Unnamed: 0,province,district,sub_district,village,approx_long,approx_lat
0,0,ACEH,ACEH SELATAN,BAKONGAN,KEUDE BAKONGAN,97.475542,2.928458
1,1,ACEH,ACEH SELATAN,BAKONGAN,UJUNG MANGKI,97.446373,2.941181
2,2,ACEH,ACEH SELATAN,BAKONGAN,UJUNG PADANG,97.493403,2.941533
3,3,ACEH,ACEH SELATAN,BAKONGAN,KAMPONG DRIEN,97.478184,2.977676
4,4,ACEH,ACEH SELATAN,BAKONGAN,DARUL IKHSAN,97.418400,2.956487


In [ ]:
df_kel['foreign_nama'] = df_kel.apply(lambda row: getParentName(row, 'name', df_kec), axis=1) # like before, get the parent name
df_kel.head()

,id,foreign,name,foreign_nama
0,1101010001,1101010,LATIUNG,TEUPAH SELATAN
1,1101010002,1101010,LABUHAN BAJAU,TEUPAH SELATAN
2,1101010003,1101010,SUAK LAMATAN,TEUPAH SELATAN
3,1101010004,1101010,ANA AO,TEUPAH SELATAN
4,1101010005,1101010,LATALING,TEUPAH SELATAN


In [ ]:
df_kel['lat'] = np.nan
df_kel['long'] = np.nan
for index, row in df_kel.iterrows():
  try:
    select = df_kec_kel[(df_kec_kel['sub_district'] == row['foreign_nama']) & (df_kec_kel['village'] == row['name'])].iloc[0]
    df_kel.loc[index, 'lat'] = select['approx_lat']
    df_kel.loc[index, 'long'] = select['approx_long']
  except:
    pass

df_kel.head()

,id,foreign,name,foreign_nama,lat,long
0,1101010001,1101010,LATIUNG,TEUPAH SELATAN,2.357417,96.452420
1,1101010002,1101010,LABUHAN BAJAU,TEUPAH SELATAN,2.391348,96.479117
2,1101010003,1101010,SUAK LAMATAN,TEUPAH SELATAN,2.353943,96.364809
3,1101010004,1101010,ANA AO,TEUPAH SELATAN,2.439100,96.450882
4,1101010005,1101010,LATALING,TEUPAH SELATAN,2.458705,96.455400


In [ ]:
del df_kel['foreign_nama']

In [ ]:
df_kel

,id,foreign,name,lat,long
0,1101010001,1101010,LATIUNG,2.357417,96.452420
1,1101010002,1101010,LABUHAN BAJAU,2.391348,96.479117
2,1101010003,1101010,SUAK LAMATAN,2.353943,96.364809
3,1101010004,1101010,ANA AO,2.439100,96.450882
4,1101010005,1101010,LATALING,2.458705,96.455400
...,...,...,...,...,...
80529,9471040003,9471040,MANDALA,-2.532777,140.720393
80530,9471040004,9471040,TRIKORA,-2.526176,140.717732
80531,9471040005,9471040,ANGKASAPURA,-2.510101,140.697874
80532,9471040007,9471040,TANJUNG RIA,-2.500108,140.709474


In [ ]:
df_kel.to_csv(KEL_OUTPUT_PATH, na_rep='null')
files.download(KEL_OUTPUT_PATH)